# Setup

In [ ]:
# imports
import polars as pl
import httpx
from pathlib import Path
import polars as pl
from sqlalchemy import create_engine
import pyarrow as pa
import pandas as pd

# Import from the _models directory and madden.py file
from sharpshooter.notebooks._models.madden import PlayerRating, RatingsResponse

In [ ]:
# Set the number of rows to display
pl.Config.set_tbl_cols(-1)  # None means no limit; you can also specify an integer.

# Set the number of columns to display
pl.Config.set_tbl_rows(-1)  # None means no limit; you can also specify an integer.

# Utilities

In [ ]:
# Create a Function that Gets the Headers of the dataframe and stores them in a list
def get_headers(df):
    headers_with_types = [(col, df[col].dtype) for col in df.columns]
    # Sort by data type first, then alphabetically within each data type
    sorted_headers = sorted(headers_with_types, key=lambda x: (str(x[1]), x[0]))
    sorted_headers_list = [col for col, dtype in sorted_headers]
    return sorted_headers, sorted_headers_list


# Madden Ratings

In [ ]:
BASE_URL = "https://ratings-api.ea.com/v2/entities"

In [ ]:
def get_madden_ratings(game_version: str, iteration: str) -> pl.DataFrame:
    url = f"{BASE_URL}/{game_version}-ratings?filter=iteration:{iteration}"
    all_ratings = []

    with httpx.Client() as client:
        response = client.get(url)
        data = response.json()
        ratings_response = RatingsResponse(**data)
        all_ratings.extend(ratings_response.docs)

        total_count = ratings_response.count
        while len(all_ratings) < total_count:
            next_url = f"{url}&limit=100&offset={len(all_ratings)}"
            response = client.get(next_url)
            data = response.json()
            ratings_response = RatingsResponse(**data)
            all_ratings.extend(ratings_response.docs)

    # Convert the list of PlayerRating objects to a list of dictionaries
    ratings_dicts = [rating.dict() for rating in all_ratings]

    # Create a Polars DataFrame from the list of dictionaries
    df = pl.DataFrame(ratings_dicts)

    # Add a fullName column
    df = df.with_columns(
        (pl.col("firstName") + " " + pl.col("lastName")).alias("fullName")
    )

    # Add a new madden_version column based on the game_version parameter
    df = df.with_columns(pl.lit(game_version).alias("madden_version"))

    # Define the game version descriptions
    game_version_descriptions = {
        "m21": "Madden 21",
        "m22": "Madden 22",
        "m23": "Madden 23",
        "m24": "Madden 24",
    }

    # Create a function to map game versions to descriptions
    def map_game_version(version):
        return game_version_descriptions.get(version, version)

    # Add the new columns to the DataFrame
    df = df.with_columns([
        pl.lit(game_version).alias("madden_version"),
        pl.col("madden_version").map_elements(map_game_version, return_dtype=pl.Utf8).alias("madden_description")
    ])

    return df

In [ ]:
madden_data = get_madden_ratings("m24", "launch-ratings")

In [ ]:
madden_data.head(10)

In [9]:
from datetime import datetime

def parse_date(date_str):
    formats = ["%Y-%m-%d", "%m/%d/%Y", "%m/%d/%y"]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).date()
        except ValueError:
            continue
    return None  # Return None if no format matches

In [ ]:
madden_data.head(10)

In [12]:
def store_to_postgres(df: pl.DataFrame, schema: str, table_name: str):
    # Database connection details
    db_username = ""
    db_password = ""
    db_host = "localhost"
    db_port = "5432"
    db_name = "sportsdata"

    # Create SQLAlchemy engine
    engine = create_engine(f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

    # Convert Polars DataFrame to Pandas DataFrame
    pandas_df = df.to_pandas()

    # Store the dataframe to the database
    pandas_df.to_sql(table_name, engine, schema=schema, if_exists="append", index=False)

    print(f"Data successfully stored in {schema}.{table_name}")

In [ ]:
store_to_postgres(madden_data, "raw", "m24__player_ratings")

In [ ]:
if __name__ == "__main__":
    madden_data = get_madden_ratings("m24", "week-5")
    madden_data = madden_data.with_columns(
    pl.col("plyrBirthdate").map_elements(parse_date, return_dtype=pl.Date).alias("plyrBirthdate")
)

# Handle any remaining null values
madden_data = madden_data.with_columns(
    pl.when(pl.col("plyrBirthdate").is_null())
    .then(pl.lit(None).cast(pl.Date))
    .otherwise(pl.col("plyrBirthdate"))
    .alias("plyrBirthdate")
)
store_to_postgres(madden_data, "raw", "m24__player_ratings")

In [ ]:
def store_madden_data_for_weeks(start_week: int, end_week: int):
    for week in range(start_week, end_week + 1):
        week_str = f"week-{week}"
        print(f"Processing {week_str}...")
        madden_data = get_madden_ratings("m24", week_str)
        
        # Handle date parsing
        madden_data = madden_data.with_columns(
            pl.col("plyrBirthdate").map_elements(parse_date, return_dtype=pl.Date).alias("plyrBirthdate")
        )

        # Handle any remaining null values
        madden_data = madden_data.with_columns(
            pl.when(pl.col("plyrBirthdate").is_null())
            .then(pl.lit(None).cast(pl.Date))
            .otherwise(pl.col("plyrBirthdate"))
            .alias("plyrBirthdate")
        )
        
        # Store the data
        table_name = f"m24__player_ratings"
        store_to_postgres(madden_data, "raw", table_name)
        print(f"Stored data for {week_str}")

if __name__ == "__main__":
    store_madden_data_for_weeks(6, 18)  # This will process weeks 5 through 10

# NFL Verse Data

In [ ]:
# Create a function to read parquet data from the NFL Verse repo
def get_nflverse_roster_data():
    url = 'https://github.com/nflverse/nflverse-data/releases/download/rosters/roster_2023.parquet'
    df = pl.read_parquet(url)
    return df


In [ ]:
# Create a function to read parquet data from the NFL Verse repo
def get_nflverse_player_data():
    url = 'https://github.com/nflverse/nflverse-data/releases/download/players/players.parquet'
    df = pl.read_parquet(url)
    return df

In [ ]:
# Create a function to read parquet data from the NFL Verse repo that accepts and argument as 'year'
def get_nflverse_depth_charts_data(year: int):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/depth_charts/depth_charts_{year}.parquet'
    df = pl.read_parquet(url)
    return df   


In [ ]:
# Create a function to read parquet data from the NFL Verse repo that accepts and argument as 'year'
def get_nflverse_weekly_rosters_data(year: int):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/weekly_rosters/roster_weekly_{year}.parquet'
    df = pl.read_parquet(url)
    return df   

In [ ]:
# Create a function to read parquet data from the NFL Verse repo that accepts and argument as 'year'
def get_nflverse_pbp_data(year: int):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.parquet'
    df = pl.read_parquet(url)
    return df   

In [ ]:
# Create a function to read parquet data from the NFL Verse repo
def get_injuries_data(year: int):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/injuries/injuries_{year}.parquet'
    df = pl.read_parquet(url)
    return df

In [ ]:
injuries_df = get_injuries_data(2024)
injuries_df

In [ ]:
# Let's convert the nflverse data to a DataFrame
roster_df = get_nflverse_roster_data()


In [ ]:
roster_df.head()

In [ ]:

player_df = get_nflverse_player_data()


In [ ]:
player_df.head(200)


In [ ]:

depth_charts_df = get_nflverse_depth_charts_data(2024)



In [ ]:
filtered_df = depth_charts_df.filter(
    (pl.col("club_code") == "MIA") & (pl.col("position") == "WR")
)

In [ ]:
filtered_df

In [ ]:

weekly_roster_df = get_nflverse_weekly_rosters_data(2024)


In [ ]:

pbp_df = get_nflverse_pbp_data(2023)

In [ ]:
weekly_roster_df.head(5)

In [ ]:
store_to_postgres(depth_charts_df, "raw", "nflverse__depth_charts")

In [ ]:
sorted_headers, sorted_headers_list = get_headers(roster_df)

In [ ]:
pbp_df.sample(20, seed = 23).write_clipboard(separator=",")

# ESPN Rankings

In [ ]:
def get_espn_rankings():
    # Get the directory of the current notebook
    notebook_dir = Path().absolute()
    
    # Navigate up to the project root and then to the data directory
    project_root = notebook_dir.parent.parent
    data_path = project_root / 'data' / 'NFL_Rankings_Complete.csv'
    
    # Check if the file exists
    if not data_path.exists():
        raise FileNotFoundError(f"The file {data_path} does not exist.")
    
    # Read the CSV file
    df = pl.read_csv(data_path)
    return df

In [ ]:
# Let's convert the ESPN Rankings data to a DataFrame
espn_df = get_espn_rankings()
espn_df.head()

# Draftkings Rankings

In [ ]:
def get_draftkings_rankings():
    # Get the directory of the current notebook
    notebook_dir = Path().absolute()
    
    # Navigate up to the project root and then to the data directory
    project_root = notebook_dir.parent.parent
    data_path = project_root / 'data' / 'DkPreDraftRankings.csv'
    
    # Check if the file exists
    if not data_path.exists():
        raise FileNotFoundError(f"The file {data_path} does not exist.")
    
    # Read the CSV file
    df = pl.read_csv(data_path)
    return df

# Test the function
draftkings_df = get_draftkings_rankings()

In [ ]:
draftkings_df = get_draftkings_rankings()
draftkings_df.head()

In [ ]:
# Create a dictionary for name mapping
name_mapping = {
    "Hollywood Brown": "Marquise Brown",
    "DJ Chark": "DJ Chark Jr.",
    # Add more mappings as needed
}

In [ ]:
# Function to apply the mapping
def map_name(name):
    return name_mapping.get(name, name)

In [ ]:
# Apply the mapping to the DraftKings dataframe
draftkings_df = draftkings_df.with_columns(
    pl.col('Name').map_elements(map_name).alias('Mapped Name')
)


In [ ]:
draftkings_df.head()

In [ ]:
def combine_rankings(draftkings_df: pl.DataFrame, espn_df: pl.DataFrame) -> pl.DataFrame:
    """
    Perform a left join on ESPN and DraftKings dataframes based on player name.
    
    Args:
    draftkings_df (pl.DataFrame): DraftKings rankings dataframe
    espn_df (pl.DataFrame): ESPN rankings dataframe

    
    Returns:
    pl.DataFrame: Combined dataframe with ESPN rankings as the base
    """
    # Rename columns to avoid conflicts and clarify source
    draftkings_df = draftkings_df.rename({
        "ADP": "DraftKings_ADP",
        "Position": "DraftKings_Position",
        "Team": "DraftKings_Team"
    })
    
    espn_df = espn_df.rename({
        "Overall Rank": "ESPN_Rank",
        "Positional Rank": "ESPN_Positional_Rank",
        "Salary Cap Value": "ESPN_Salary_Cap_Value"
    })
    

    
    # Perform the left join
    combined_df = draftkings_df.join(
        espn_df,
        left_on="Mapped Name",
        right_on="Player Name",
        how="left"
    )
    
    # Drop the duplicate "Name" column from DraftKings
    # combined_df = combined_df.drop("Name")
    
    return combined_df

# Example usage:
# espn_df = pl.read_csv("espn_rankings_sample.csv")
# draftkings_df = pl.read_csv("draftkings_rankings_sample.csv")
result = combine_rankings(draftkings_df, espn_df)
# print(result)

In [ ]:
result.write_csv("combined_rankings.csv")

In [ ]:
dk_to_madden_name_mapping = {
    "Deebo Samuel Sr.": "Deebo Samuel Sr",
    "Marvin Harrison Jr.": "Marvin Harrison Jr",
    "Travis Etienne Jr." : "Travis Etienne Jr",
    "DJ Moore": "D.J. Moore",
    "DK Metcalf": "D.K. Metcalf",
    "Michael Pittman Jr.": "Michael Pittman Jr",
    "Hollywood Brown": "Marquise Brown",
    "Brian Thomas Jr.": "Brian Thomas Jr",
    "Brian Robinson Jr.": "Brian Robinson Jr",
    "Marvin Mims Jr.": "Marvin Mims Jr",
    "Tyrone Tracy Jr.": "Tyrone Tracy Jr",
    "DJ Chark": "DJ Chark Jr",
    "AJ Dillon": "A.J. Dillon",
    "Chris Rodriguez Jr.": "Chris Rodriguez Jr",
    "Odell Beckham Jr.": "Odell Beckham Jr",
    "Michael Penix Jr.": "Michael Penix Jr",
    "DeMario Douglas": "Demario Douglas",
    # Add more mappings as needed
}

def combine_madden_with_rankings(combined_df: pl.DataFrame, madden_df: pl.DataFrame) -> pl.DataFrame:
    """
    Perform a left join on the Combined Ranks and Madden dataframes based on player name, team, and position.
    
    Args:
    combined_df (pl.DataFrame): Combined rankings dataframe
    madden_df (pl.DataFrame): Madden ratings dataframe

    Returns:
    pl.DataFrame: Combined dataframe with Combined Rankings as the base
    """
    # Apply the name mapping to the combined_df
    combined_df = combined_df.with_columns(
        pl.when(pl.col("Name").is_in(dk_to_madden_name_mapping.keys()))
          .then(pl.col("Name").replace(dk_to_madden_name_mapping))
          .otherwise(pl.col("Name"))
          .alias("Mapped_Name")
    )

    # Rename columns to avoid conflicts and clarify source
    combined_df = combined_df.rename({
        "Mapped_Name": "Join_Name"
    })
    
    madden_df = madden_df.rename({
        "fullName": "Join_Name",
    })   

    # Perform the left join
    combined_madden_df = combined_df.join(
        madden_df,
        on=["Join_Name"],
        how="left"
    )
    
    # Select only the required columns from madden_df
    madden_df_selected = combined_madden_df.select([
        "ID",
        "Name",
        "DraftKings_Position",
        "DraftKings_ADP",
        "DraftKings_Team",
        "ESPN_Rank",
        "ESPN_Positional_Rank",
        "overallRating"
    ])    
    
    return madden_df_selected


In [ ]:
combined_madden_df_t = combine_madden_with_rankings(result, df)

In [ ]:
combined_madden_df_t.head(5)

In [ ]:
# Filter the Madden Dataframe for Players with the fullNameForSearch *like* Marvin Harrison
combined_madden_df_t.filter(pl.col("Name").str.contains("DeMario"))

In [ ]:
combined_madden_df_t.write_csv("combined_madden_rankings.csv")

In [ ]:
# Create a Dictionary between the 'fullName' and 'Name' columns
dk_to_madden_name_mapping = {
    "Deebo Samuel Sr.": "Deebo Samuel Sr",
    "Marvin Harrison Jr.": "Marvin Harrison Jr",
    # Add more mappings as needed
}

# Create a Function to apply the mapping
def map_name(name):
    return name_mapping.get(name, name)

In [ ]:
# Find Marvin Harrison in the df Dataframe

df.filter(pl.col("fullName").str.contains("Deebo"))

In [ ]:
combined_madden_df_t.head(100)

In [ ]:
# Let's add a column to the df dataframe that combines firstName and lastName Columns and place it first
madden25_df = df.with_columns([
    pl.col("firstName") + " " + pl.col("lastName").alias("fullName")
])

In [ ]:
# Filter the Madden Dataframe for Players with the fullNameForSearch *like* Marvin Harrison
madden_df.filter(pl.col("fullNameForSearch").str.contains("Marvin Harrison"))


# madden_df.filter(pl.col("fullNameForSearch") == "Marvin Harrison Jr")

In [7]:
import httpx
from pydantic import BaseModel, Field
from typing import List, Optional, Union, Dict

BASE_URL = "https://drop-api.ea.com/rating/madden-nfl"

class Team(BaseModel):
    id: int
    label: str
    imageUrl: str
    isPopular: bool

class PositionType(BaseModel):
    id: str
    name: str

class Position(BaseModel):
    id: str
    shortLabel: str
    label: str
    positionType: PositionType

class Archetype(BaseModel):
    id: str
    label: str

class Iteration(BaseModel):
    id: str
    label: str

class NumericStat(BaseModel):
    value: float
    diff: int

class RunningStyleStat(BaseModel):
    value: str
    diff: int

class AbilityType(BaseModel):
    id: str
    label: str
    imageUrl: str
    iconUrl: str

class Ability(BaseModel):
    id: str
    label: str
    description: str
    imageUrl: str
    type: AbilityType

class PlayerRating(BaseModel):
    id: int
    overallRating: int
    firstName: str
    lastName: str
    birthdate: str
    height: int
    weight: int
    college: str
    handedness: int
    age: int
    jerseyNum: int
    yearsPro: int
    playerAbilities: List[Ability]
    avatarUrl: Optional[str]
    archetype: Optional[Archetype]
    team: Team
    position: Position
    iteration: Iteration
    stats: Dict[str, Union[NumericStat, RunningStyleStat]]

class RatingsResponse(BaseModel):
    items: List[PlayerRating]
    totalItems: int

def get_madden_ratings(locale: str = "en", iteration: str = "2-week-1") -> List[PlayerRating]:
    """
    Fetches Madden ratings based on the specified locale and iteration.

    Args:
        locale (str): The locale for the ratings. Defaults to "en".
        iteration (str): The iteration of the ratings. Defaults to "1-base".

    Returns:
        List[PlayerRating]: A list of player ratings.
    """
    url = f"{BASE_URL}?locale={locale}&iteration={iteration}"
    all_ratings = []

    with httpx.Client() as client:
        response = client.get(url)
        response.raise_for_status()
        data = response.json()
        ratings_response = RatingsResponse(**data)
        all_ratings.extend(ratings_response.items)

        total_count = ratings_response.totalItems
        while len(all_ratings) < total_count:
            next_url = f"{url}&limit=100&offset={len(all_ratings)}"
            response = client.get(next_url)
            response.raise_for_status()
            data = response.json()
            ratings_response = RatingsResponse(**data)
            all_ratings.extend(ratings_response.items)

    print(f"Total items fetched: {len(all_ratings)}")
    print(f"Expected total items: {total_count}")

    return all_ratings

def create_madden_nfl_dataframe() -> pl.DataFrame:
    """
    Creates a Polars DataFrame from Madden NFL ratings data.

    Returns:
        pl.DataFrame: A DataFrame containing Madden NFL player ratings.
    """
    ratings = get_madden_ratings(iteration="5-week-4")
    
    # Convert Pydantic models to dictionaries
    data = [rating.dict() for rating in ratings]
    
    # Create DataFrame
    df = pl.DataFrame(data)
    
    # Add fullName column
    df = df.with_columns([
        (pl.col("firstName") + " " + pl.col("lastName")).alias("fullName")
    ])
    
    # Flatten nested structures
    df = df.with_columns([
        pl.col("team").struct.field("id").alias("team_id"),
        pl.col("team").struct.field("label").alias("team_label"),
        pl.col("team").struct.field("imageUrl").alias("team_imageUrl"),
        pl.col("team").struct.field("isPopular").alias("team_isPopular"),
        pl.col("position").struct.field("id").alias("position_id"),
        pl.col("position").struct.field("shortLabel").alias("position_shortLabel"),
        pl.col("position").struct.field("label").alias("position_label"),
        pl.col("position").struct.field("positionType").struct.field("id").alias("position_type_id"),
        pl.col("position").struct.field("positionType").struct.field("name").alias("position_type_name"),
        pl.col("iteration").struct.field("id").alias("iteration_id"),
        pl.col("iteration").struct.field("label").alias("iteration_label"),
        pl.col("archetype").struct.field("id").alias("archetype_id"),
        pl.col("archetype").struct.field("label").alias("archetype_label"),
    ])

    # Flatten stats
    stat_columns = df.select(pl.col("stats")).to_series().struct.fields
    for stat in stat_columns:
        df = df.with_columns([
            pl.col("stats").struct.field(stat).struct.field("value").alias(f"{stat}_rating")
        ])

    # Drop original nested columns
    df = df.drop(["team", "position", "iteration", "stats", "playerAbilities", "archetype"])

    # Convert data types
    date_columns = ["birthdate"]
    int_columns = ["id", "overallRating", "height", "weight", "handedness", "age", "jerseyNum", "yearsPro", "team_id"]
    float_columns = [col for col in df.columns if col.endswith("_rating") and col != "runningStyle_rating"]
    bool_columns = ["team_isPopular"]

    for col in date_columns:
        df = df.with_columns([
            pl.col(col)
            .str.strptime(pl.Date, format="%Y-%m-%d", strict=False)  # Changed 'fmt' to 'format'
            .fill_null(
                pl.col(col).str.strptime(pl.Date, format="%m/%d/%Y", strict=False)  # Changed 'fmt' to 'format'
            )
            .alias(col)
        ])

    for col in int_columns:
        df = df.with_columns(pl.col(col).cast(pl.Int64))

    for col in float_columns:
        df = df.with_columns(pl.col(col).cast(pl.Float64))

    for col in bool_columns:
        df = df.with_columns(pl.col(col).cast(pl.Boolean))

    print(f"DataFrame shape: {df.shape}")
    return df

# Example usage
if __name__ == "__main__":
    df = create_madden_nfl_dataframe()
    # store_to_postgres(df, "raw", "m25__player_ratings")

Total items fetched: 1932
Expected total items: 1932
DataFrame shape: (1932, 82)


In [8]:
df.head(1)

id,overallRating,firstName,lastName,birthdate,height,weight,college,handedness,age,jerseyNum,yearsPro,avatarUrl,fullName,team_id,team_label,team_imageUrl,team_isPopular,position_id,position_shortLabel,position_label,position_type_id,position_type_name,iteration_id,iteration_label,archetype_id,archetype_label,acceleration_rating,agility_rating,jumping_rating,stamina_rating,strength_rating,awareness_rating,bCVision_rating,blockShedding_rating,breakSack_rating,breakTackle_rating,carrying_rating,catchInTraffic_rating,catching_rating,changeOfDirection_rating,deepRouteRunning_rating,finesseMoves_rating,hitPower_rating,impactBlocking_rating,injury_rating,jukeMove_rating,kickAccuracy_rating,kickPower_rating,kickReturn_rating,leadBlock_rating,manCoverage_rating,mediumRouteRunning_rating,overall_rating,passBlock_rating,passBlockFinesse_rating,passBlockPower_rating,playAction_rating,playRecognition_rating,powerMoves_rating,press_rating,pursuit_rating,release_rating,runBlock_rating,runBlockFinesse_rating,runBlockPower_rating,runningStyle_rating,shortRouteRunning_rating,spectacularCatch_rating,speed_rating,spinMove_rating,stiffArm_rating,tackle_rating,throwAccuracyDeep_rating,throwAccuracyMid_rating,throwAccuracyShort_rating,throwOnTheRun_rating,throwPower_rating,throwUnderPressure_rating,toughness_rating,trucking_rating,zoneCoverage_rating
i64,i64,str,str,date,i64,i64,str,i64,i64,i64,i64,str,str,i64,str,str,bool,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
12556,99,"""Christian""","""McCaffrey""",1996-06-07,71,205,"""Stanford""",1,28,23,7,"""https://ratings-images-prod.pu…","""Christian McCaffrey""",15,"""San Francisco 49ers""","""https://drop-assets.ea.com/ima…",false,"""HB""","""HB""","""Halfback""","""offense""","""Offense""","""5-week-4""","""Week 4 Ratings""","""HB_ReceivingBack""","""Receiving Back - HB""",93.0,95.0,90.0,93.0,74.0,99.0,97.0,32.0,38.0,90.0,96.0,72.0,83.0,93.0,67.0,10.0,27.0,45.0,87.0,94.0,23.0,29.0,88.0,47.0,12.0,76.0,99.0,64.0,32.0,36.0,31.0,27.0,10.0,15.0,44.0,76.0,38.0,22.0,22.0,"""None""",83.0,75.0,91.0,87.0,80.0,36.0,33.0,41.0,45.0,47.0,52.0,37.0,93.0,65.0,17.0


In [7]:
if __name__ == "__main__":
    madden_data = create_madden_nfl_dataframe()
    madden_data = madden_data.with_columns(
    pl.col("plyrBirthdate").map_elements(parse_date, return_dtype=pl.Date).alias("plyrBirthdate")
)

# Handle any remaining null values
madden_data = madden_data.with_columns(
    pl.when(pl.col("plyrBirthdate").is_null())
    .then(pl.lit(None).cast(pl.Date))
    .otherwise(pl.col("plyrBirthdate"))
    .alias("plyrBirthdate")
)
store_to_postgres(madden_data, "raw", "m25__player_ratings")

Total items fetched: 2357
Expected total items: 2357
DataFrame shape: (2357, 82)


NameError: name 'parse_date' is not defined

In [ ]:
# Create a Column that concatenates the 'firstName' and 'lastName' columns
madden25_df = madden25_df.with_columns([
    pl.col("firstName") + " " + pl.col("lastName").alias("fullName")
])

In [ ]:
if __name__ == "__main__":
    madden_data = create_madden_nfl_dataframe()
    madden_data = madden_data.with_columns(
    pl.col("plyrBirthdate").map_elements(parse_date, return_dtype=pl.Date).alias("plyrBirthdate")
)

# Handle any remaining null values
madden_data = madden_data.with_columns(
    pl.when(pl.col("plyrBirthdate").is_null())
    .then(pl.lit(None).cast(pl.Date))
    .otherwise(pl.col("plyrBirthdate"))
    .alias("plyrBirthdate")
)
store_to_postgres(madden_data, "raw", "m25__player_ratings")